In [14]:
import os
import sys
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from scipy.io import loadmat
from time import time

In [2]:
from pyod.models.pca import PCA
from pyod.models.mcd import MCD
from pyod.models.ocsvm import OCSVM
from pyod.models.lof import LOF
from pyod.models.cblof import CBLOF
from pyod.models.knn import KNN
from pyod.models.hbos import HBOS
from pyod.models.abod import ABOD
from pyod.models.iforest import IForest
from pyod.models.feature_bagging import FeatureBagging

In [3]:
from pyod.utils.utility import standardizer
from pyod.utils.utility import precision_n_scores
from sklearn.metrics import roc_auc_score

In [10]:
mat_file_list = os.listdir('Dataset-2/Anamoly_detec_data')
mat_file_list.remove('README.md')

In [11]:
len(mat_file_list)

17

In [12]:
df_columns=['Data','#Sample','#Dimensions',
            'Outlier Perc','PCA','MCD','OCSVM','LOF','CBLOF','KNN','HBOS','ABOD','IFOREST','FEATUREBAGGING']

In [16]:
roc_df=pd.DataFrame(columns=df_columns)
prn_df=pd.DataFrame(columns=df_columns)
time_df=pd.DataFrame(columns=df_columns)
print(roc_df,prn_df,time_df)

Empty DataFrame
Columns: [Data, #Sample, #Dimensions, Outlier Perc, PCA, MCD, OCSVM, LOF, CBLOF, KNN, HBOS, ABOD, IFOREST, FEATUREBAGGING]
Index: [] Empty DataFrame
Columns: [Data, #Sample, #Dimensions, Outlier Perc, PCA, MCD, OCSVM, LOF, CBLOF, KNN, HBOS, ABOD, IFOREST, FEATUREBAGGING]
Index: [] Empty DataFrame
Columns: [Data, #Sample, #Dimensions, Outlier Perc, PCA, MCD, OCSVM, LOF, CBLOF, KNN, HBOS, ABOD, IFOREST, FEATUREBAGGING]
Index: []


In [19]:
randomstate = np.random.RandomState(42)

for file in mat_file_list:
    print('\n... Processing', file,'.....')
    mat = loadmat('Dataset-2/Anamoly_detec_data/'+ file)
    
    X = mat['X']
    y = mat['y'].ravel()
    outliers_fraction = np.count_nonzero(y)/ len(y)
    outliers_pct = round(outliers_fraction * 100, ndigits= 4)
    
    #construct container for saving results
    roc_list = [file[:-4], X.shape[0], X.shape[1], outliers_pct]
    prn_list = [file[:-4], X.shape[0], X.shape[1], outliers_pct]
    time_list = [file[:-4], X.shape[0], X.shape[1], outliers_pct]
    
    X_train, X_test, y_train, y_test = train_test_split(X, y , 
                                                        test_size = 0.4, random_state= randomstate)
    
    
    # standardizing data for processing
    X_train_norm, X_test_norm = standardizer(X_train, X_test)
    
    classifiers = {'ABOD' : ABOD(contamination=outliers_fraction),
                  'CBLOF' : CBLOF(contamination=outliers_fraction, check_estimator= False, random_state=randomstate),
                  'Feature_BAgging' : FeatureBagging(contamination= outliers_fraction, random_state=randomstate),
                  'HBOS': HBOS(contamination=outliers_fraction), 
                  'IsolationForest': IForest(contamination=outliers_fraction, random_state=randomstate),
                  'KNN' : KNN(contamination=outliers_fraction),
                  'LOF': LOF(contamination=outliers_fraction),
                  'MCD' : MCD(contamination= outliers_fraction,random_state=randomstate), 
                  'OCSVM' : OCSVM(contamination= outliers_fraction), 
                  'PCA' : PCA(contamination= outliers_fraction,random_state=randomstate)}
    
    
    for clf_name, clf in classifiers.items():
        t0 = time()
        clf.fit(X_train_norm)
        test_scores = clf.decision_function(X_test_norm)
        t1 = time()
        duration = round(t1 - t0, ndigits=4)
        time_list.append(duration)

        roc = round(roc_auc_score(y_test, test_scores), ndigits=4)
        prn = round(precision_n_scores(y_test, test_scores), ndigits=4)

        print('{clf_name} ROC:{roc}, precision @ rank n:{prn}, '
             'execution time: {duration}s'.format(
           clf_name=clf_name, roc=roc, prn=prn, duration=duration))

        roc_list.append(roc)
        prn_list.append(prn)

    temp_df = pd.DataFrame(time_list).transpose()
    temp_df.columns = df_columns
    time_df = pd.concat([time_df, temp_df], axis=0)

    temp_df = pd.DataFrame(roc_list).transpose()
    temp_df.columns = df_columns
    roc_df = pd.concat([roc_df, temp_df], axis=0)

    temp_df = pd.DataFrame(prn_list).transpose()
    temp_df.columns = df_columns
    prn_df = pd.concat([prn_df, temp_df], axis=0)


... Processing pima.mat .....
ABOD ROC:0.665, precision @ rank n:0.4314, execution time: 0.1323s


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:938: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 0.25.
  warnings.warn("'n_jobs' was deprecated in version 0.23 and will be"


CBLOF ROC:0.6724, precision @ rank n:0.4608, execution time: 0.0771s
Feature_BAgging ROC:0.6551, precision @ rank n:0.4608, execution time: 0.0825s
HBOS ROC:0.7029, precision @ rank n:0.5392, execution time: 0.0035s
IsolationForest ROC:0.6733, precision @ rank n:0.4804, execution time: 0.3401s
KNN ROC:0.7123, precision @ rank n:0.5, execution time: 0.0274s
LOF ROC:0.6621, precision @ rank n:0.4706, execution time: 0.0081s
MCD ROC:0.6942, precision @ rank n:0.4706, execution time: 0.0556s
OCSVM ROC:0.6423, precision @ rank n:0.4412, execution time: 0.0079s
PCA ROC:0.6444, precision @ rank n:0.451, execution time: 0.0019s

... Processing cardio.mat .....
ABOD ROC:0.5535, precision @ rank n:0.2623, execution time: 0.3766s
CBLOF ROC:0.8089, precision @ rank n:0.5246, execution time: 0.0951s


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:938: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 0.25.
  warnings.warn("'n_jobs' was deprecated in version 0.23 and will be"


Feature_BAgging ROC:0.5089, precision @ rank n:0.1148, execution time: 0.5542s
HBOS ROC:0.8653, precision @ rank n:0.4262, execution time: 0.0074s
IsolationForest ROC:0.9118, precision @ rank n:0.4262, execution time: 0.3956s
KNN ROC:0.7064, precision @ rank n:0.3607, execution time: 0.1158s
LOF ROC:0.5173, precision @ rank n:0.1148, execution time: 0.067s


...

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:938: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 0.25.
  warnings.warn("'n_jobs' was deprecated in version 0.23 and will be"


Feature_BAgging ROC:0.4067, precision @ rank n:0.0938, execution time: 4.4473s
HBOS ROC:0.982, precision @ rank n:0.7188, execution time: 0.0162s
IsolationForest ROC:0.9956, precision @ rank n:0.875, execution time: 0.5784s
KNN ROC:0.9551, precision @ rank n:0.375, execution time: 0.6555s
LOF ROC:0.4001, precision @ rank n:0.0625, execution time: 0.5431s
MCD ROC:0.9953, precision @ rank n:0.625, execution time: 2.4633s
OCSVM ROC:0.9979, precision @ rank n:0.9062, execution time: 0.8356s
PCA ROC:0.9835, precision @ rank n:0.8438, execution time: 0.0159s

... Processing vertebral.mat .....
ABOD ROC:0.3797, precision @ rank n:0.0, execution time: 0.0423s
CBLOF ROC:0.4182, precision @ rank n:0.0, execution time: 0.0499s
Feature_BAgging ROC:0.3561, precision @ rank n:0.0909, execution time: 0.0309s
HBOS ROC:0.3182, precision @ rank n:0.0, execution time: 0.0029s


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:938: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 0.25.
  warnings.warn("'n_jobs' was deprecated in version 0.23 and will be"


IsolationForest ROC:0.3626, precision @ rank n:0.0909, execution time: 0.3004s
KNN ROC:0.3487, precision @ rank n:0.0, execution time: 0.0087s
LOF ROC:0.354, precision @ rank n:0.0909, execution time: 0.0027s
MCD ROC:0.4866, precision @ rank n:0.1818, execution time: 0.0476s
OCSVM ROC:0.3925, precision @ rank n:0.0909, execution time: 0.0017s
PCA ROC:0.3647, precision @ rank n:0.0, execution time: 0.0019s


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/covariance/_robust_covariance.py:166: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-23.882019415506797 > -43.199021505473233). You may want to try with a higher value of support_fraction (current value: 0.528).
  warnings.warn("Determinant has increased; this should not happen: "
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/covariance/_robust_covariance.py:166: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-24.267044435718223 > -42.928224907436515). You may want to try with a higher value of support_fraction (current value: 0.528).
  warnings.warn("Determinant has increased; this should not happen: "
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/covariance/_robust_covariance.py:166: RuntimeWarning: Determinant has increa

In [36]:
roc_df = roc_df.reset_index(drop=True)

In [37]:
roc_df

,Data,#Sample,#Dimensions,Outlier Perc,PCA,MCD,OCSVM,LOF,CBLOF,KNN,HBOS,ABOD,IFOREST,FEATUREBAGGING
0,pima,768,8,34.8958,0.665,0.6724,0.6551,0.7029,0.6733,0.7123,0.6621,0.6942,0.6423,0.6444
1,cardio,1831,21,9.6122,0.5535,0.8089,0.5089,0.8653,0.9118,0.7064,0.5173,0.8231,0.9347,0.9517
2,mnist,7603,100,9.2069,0.7674,0.8204,0.6871,0.5792,0.8025,0.8434,0.6832,0.8976,0.8569,0.8536
3,arrhythmia,452,274,14.6018,0.7628,0.7806,0.7772,0.8345,0.8236,0.7806,0.7856,0.7712,0.7744,0.7834
4,pendigits,6870,16,2.2707,0.6481,0.9691,0.3709,0.9138,0.9407,0.7634,0.3709,0.8273,0.9304,0.9349
5,shuttle,49097,9,7.1511,0.6215,0.5709,0.4425,0.9828,0.9969,0.6479,0.5193,0.9896,0.9896,0.9875
6,letter,1600,32,6.25,0.8764,0.781,0.8505,0.564,0.6237,0.8453,0.8548,0.8032,0.5727,0.5004
7,musk,3062,166,3.1679,0.2342,1,0.6652,1,1,0.815,0.6575,0.9999,1,1
8,vowels,1456,12,3.4341,0.9086,0.863,0.9281,0.5922,0.6879,0.95,0.9173,0.834,0.6755,0.476
9,optdigits,5216,64,2.8758,0.4962,0.7527,0.4248,0.8801,0.7595,0.3783,0.4261,0.3925,0.4658,0.4947


In [39]:
prn_df.reset_index(drop=True)

,Data,#Sample,#Dimensions,Outlier Perc,PCA,MCD,OCSVM,LOF,CBLOF,KNN,HBOS,ABOD,IFOREST,FEATUREBAGGING
0,pima,768,8,34.8958,0.4314,0.4608,0.4608,0.5392,0.4804,0.5,0.4706,0.4706,0.4412,0.451
1,pima,768,8,34.8958,0.4314,0.4608,0.4608,0.5392,0.4804,0.5,0.4706,0.4706,0.4412,0.451
2,cardio,1831,21,9.6122,0.2623,0.5246,0.1148,0.4262,0.4262,0.3607,0.1148,0.4098,0.4754,0.5082
3,mnist,7603,100,9.2069,0.3148,0.3593,0.2741,0.1037,0.2741,0.4,0.2926,0.4963,0.3704,0.363
4,arrhythmia,452,274,14.6018,0.3462,0.4615,0.3462,0.5769,0.5385,0.4615,0.4615,0.3077,0.5,0.5385
5,pendigits,6870,16,2.2707,0.0606,0.303,0.0303,0.2576,0.2727,0.0758,0.0303,0.0909,0.2879,0.2879
6,shuttle,49097,9,7.1511,0.1896,0.2408,0.0767,0.994,0.9418,0.2131,0.1229,0.7379,0.9531,0.9503
7,letter,1600,32,6.25,0.3478,0.1957,0.3261,0.087,0.087,0.2609,0.3478,0.1957,0.087,0.0435
8,musk,3062,166,3.1679,0.1892,1,0.4054,1,1,0.3784,0.4054,0.973,1,1
9,vowels,1456,12,3.4341,0.4211,0.1053,0.2632,0.1053,0.1053,0.3684,0.2632,0.1579,0.1579,0.1053


In [40]:
time_df.reset_index(drop=True)

,Data,#Sample,#Dimensions,Outlier Perc,PCA,MCD,OCSVM,LOF,CBLOF,KNN,HBOS,ABOD,IFOREST,FEATUREBAGGING
0,pima,768,8,34.8958,0.1271,0.0758,0.0782,0.0036,0.3332,0.0281,0.0078,0.0558,0.0081,0.0021
1,pima,768,8,34.8958,0.1323,0.0771,0.0825,0.0035,0.3401,0.0274,0.0081,0.0556,0.0079,0.0019
2,cardio,1831,21,9.6122,0.3766,0.0951,0.5542,0.0074,0.3956,0.1158,0.067,0.6834,0.0836,0.005
3,mnist,7603,100,9.2069,5.0315,0.4914,27.1953,0.0474,0.9971,4.0263,3.9825,2.8684,3.2887,0.0918
4,arrhythmia,452,274,14.6018,0.1238,0.0967,0.3847,0.0704,0.4093,0.0594,0.0465,0.7914,0.0347,0.0553
5,pendigits,6870,16,2.2707,1.4382,0.1612,2.9555,0.009,0.574,0.467,0.383,2.0536,0.7441,0.0076
6,shuttle,49097,9,7.1511,11.3306,0.3586,32.8587,0.016,1.7731,5.4693,6.997,9.1099,37.0685,0.0223
7,letter,1600,32,6.25,0.3278,0.0997,0.5145,0.01,0.3708,0.1003,0.0611,1.3893,0.0796,0.0067
8,musk,3062,166,3.1679,1.616,0.1631,8.3376,0.057,0.6831,1.1672,1.0578,10.5839,0.8198,0.1016
9,vowels,1456,12,3.4341,0.2387,0.0719,0.2016,0.0047,0.3542,0.0596,0.0246,0.7193,0.0308,0.0031
